In [1]:
import os
if not os.path.exists('data'):
    os.makedirs('data')

# Designing to Mitigate Urban Heat Island

The Urban Heat Island effect refers to the environmental conditions in most urban or metropolitan areas being significantly warmer than surrounding rural areas because of human activities. 

In [2]:
from dragonfly.uwg.districtpar import TrafficPar, PavementPar
from dragonfly.uwg.regionpar import RefEPWSitePar, BoundaryLayerPar
from dragonfly.uwg.typologypar import TypologyPar
from dragonfly.district import District
from dragonfly.uwg.run import RunManager
from dragonfly.typology import Typology

# epw_file = os.path.abspath('data/ESP_MALAGA-AP_084820_IW2.epw')

def run_parameters(grass_cov=0.0, tree_cov=0.5, glz_ratio=0.3,
                   roof_albedo=0.2, wall_albedo=0.15, pav_albedo=0.1,
                   traffic_heat=4, epw_path='data/ESP_MALAGA-AP_084820_IW2.epw', urban_epw_path=None):
    
    tp = TrafficPar(traffic_heat)
    pp = PavementPar(albedo=pav_albedo)
    typ_p = TypologyPar(wall_albedo=wall_albedo, roof_albedo=roof_albedo, roof_veg_fraction=grass_cov)
    
    ref_epw_site_par = RefEPWSitePar(average_obstacle_height=None,
                                 vegetation_coverage=0.7,
                                 temp_measure_height=None,
                                 wind_measure_height=None)

    boundary_layer_par = BoundaryLayerPar()

    # Climate zone
    climate_zone = '3A'

    # Terrain setup
    site_area = 160000
    # terrain_char_len = 300

#     terrain = Terrain(area=terrain_area)
    
    # Typologies setup
    # Need to setup residential and retail

    resi_dict = {
        'average_height': 24,
        'footprint_area': 52000,
        'facade_area': 221000,
        'bldg_program': 'MidRiseApartment',
        'bldg_age': 'Pre1980s',
        'fract_heat_to_canyon': 0.7,
        'glz_ratio': glz_ratio,
        'floor_area': 391000
    }

    residential = Typology(average_height=resi_dict['average_height'],
                           footprint_area=resi_dict['footprint_area'],
                           facade_area=resi_dict['facade_area'],
                           bldg_program=resi_dict['bldg_program'],
                           bldg_era=resi_dict['bldg_age'],
                           glz_ratio=resi_dict['glz_ratio'],
                           floor_area=resi_dict['floor_area'],
                           uwg_parameters=typ_p)

    # residential.wall_albedo =
    # residential.roof_albedo =
    residential.roof_veg_fraction = grass_cov

    sm_retail = {
        'average_height': 8,
        'footprint_area': 2256,
        'facade_area': 2424,
        'bldg_program': 'StandAloneRetail',
        'bldg_age': 'Pre1980s',
        'fract_heat_to_canyon': 0.7,
        'glz_ratio': glz_ratio,
        'floor_area': 4511
    }

    small_retail = Typology(average_height=sm_retail['average_height'],
                            footprint_area=sm_retail['footprint_area'],
                            facade_area=sm_retail['facade_area'],
                            bldg_program=sm_retail['bldg_program'],
                            bldg_era=sm_retail['bldg_age'],
                            glz_ratio=sm_retail['glz_ratio'],
                            floor_area=sm_retail['floor_area'],
                            uwg_parameters=typ_p)
        
    
    
    district = District(building_typologies=[residential, small_retail], 
                        site_area=site_area, climate_zone=climate_zone,
                        tree_coverage_ratio=tree_cov, grass_coverage_ratio=grass_cov,
                        traffic_parameters=tp, pavement_parameters=pp)
    
    rm = RunManager(epw_path, district)
    
    if urban_epw_path == None:
        urban_epw_path= 'grass_cov_{}_tree_cov_{}_glz_ratio_{}_roof_albedo_{}_wall_albedo_{}_pav_albedo_{}_traffic_heat_{}_.epw'\
            .format(grass_cov, tree_cov, glz_ratio, roof_albedo, wall_albedo, pav_albedo, traffic_heat)
    
    urban_epw_path = 'data/{}'.format(urban_epw_path)
    
    rm.run(urban_epw_path)

## Note:
I noticed a bug in the UWG package which I have patched [here](https://github.com/ladybug-tools/uwg/pull/89). Be sure to either make the patch yourself or wait for the PR to be merged and the deployment to be made to PyPi and then update your uwg package to the latest version.

In [3]:
# %%timeit
run_parameters(urban_epw_path='baseline.epw')


Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'baseline.epw' is generated at data.


In [9]:
parameters = {
    "grass_cov": [0, 0.1, 0.3],
    "tree_cov": [0.05, 0.1, 0.15],
    "glz_ratio": [0.3, 0.2, 0.4],
#     "roof_albedo": [0.2, 0.25, 0.3],
    "wall_albedo": [0.15, 0.2, 0.25],
    "pav_albedo": [0.1, 0.15, 0.2],
    "traffic_heat": [4, 2]
    }

In [13]:
import itertools

params = [[0, 0.1, 0.3],
    [0.05, 0.1, 0.15],
    [0.3, 0.2, 0.4],
    [0.2, 0.25, 0.3],
    [0.15, 0.2, 0.25],
    [0.1, 0.15, 0.2],
    [4, 2]]


permutations = list(itertools.product(*params))

In [14]:
len(permutations)

1458

## Note:
I was lazy about this and didn't apply any fancy parallel processing and such so this took roughly 7 hours to run. Multithreading will cut the time for this dramatically. Run at your own risk!

In [11]:
# %%timeit 
for permutation in permutations:
    run_parameters(permutation[0], permutation[1], permutation[2], 0.2, permutation[3], permutation[4], permutation[5])


Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0_tree_cov_0.05_glz_ratio_0.3_roof_albedo_0.2_wall_albedo_0.15_pav_albedo_0.1_traffic_heat_4_' is generated at data.

Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0_tree_cov_0.05_glz_ratio_0.3_roof_albedo_0.2_wall_albedo_0.15_pav_albedo_0.1_traffic_heat_2_' is generated at data.

Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0_tree_cov_0.05_glz_ratio_0.3_roof_albedo_0.2_wall_albedo_0.15_pav_albedo_0.15_traffic_heat_4_' is generated at data.

Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0_tree_cov_0.05_glz_ratio_0.3_roof_albedo_0.2_wall_albedo_0.15_pav_albedo_0.15_traffic_heat_2_' is generated at data.

Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0_tree_cov_0.05_glz_ratio_0.3_


Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0_tree_cov_0.05_glz_ratio_0.4_roof_albedo_0.2_wall_albedo_0.15_pav_albedo_0.15_traffic_heat_4_' is generated at data.

Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0_tree_cov_0.05_glz_ratio_0.4_roof_albedo_0.2_wall_albedo_0.15_pav_albedo_0.15_traffic_heat_2_' is generated at data.

Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0_tree_cov_0.05_glz_ratio_0.4_roof_albedo_0.2_wall_albedo_0.15_pav_albedo_0.2_traffic_heat_4_' is generated at data.

Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0_tree_cov_0.05_glz_ratio_0.4_roof_albedo_0.2_wall_albedo_0.15_pav_albedo_0.2_traffic_heat_2_' is generated at data.

Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0_tree_cov_0.05_glz_ratio_0.4_


Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0_tree_cov_0.1_glz_ratio_0.2_roof_albedo_0.2_wall_albedo_0.15_pav_albedo_0.2_traffic_heat_4_' is generated at data.

Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0_tree_cov_0.1_glz_ratio_0.2_roof_albedo_0.2_wall_albedo_0.15_pav_albedo_0.2_traffic_heat_2_' is generated at data.

Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0_tree_cov_0.1_glz_ratio_0.2_roof_albedo_0.2_wall_albedo_0.2_pav_albedo_0.1_traffic_heat_4_' is generated at data.

Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0_tree_cov_0.1_glz_ratio_0.2_roof_albedo_0.2_wall_albedo_0.2_pav_albedo_0.1_traffic_heat_2_' is generated at data.

Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0_tree_cov_0.1_glz_ratio_0.2_roof_albe


Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0_tree_cov_0.15_glz_ratio_0.3_roof_albedo_0.2_wall_albedo_0.2_pav_albedo_0.1_traffic_heat_4_' is generated at data.

Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0_tree_cov_0.15_glz_ratio_0.3_roof_albedo_0.2_wall_albedo_0.2_pav_albedo_0.1_traffic_heat_2_' is generated at data.

Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0_tree_cov_0.15_glz_ratio_0.3_roof_albedo_0.2_wall_albedo_0.2_pav_albedo_0.15_traffic_heat_4_' is generated at data.

Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0_tree_cov_0.15_glz_ratio_0.3_roof_albedo_0.2_wall_albedo_0.2_pav_albedo_0.15_traffic_heat_2_' is generated at data.

Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0_tree_cov_0.15_glz_ratio_0.3_roof


Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0_tree_cov_0.15_glz_ratio_0.4_roof_albedo_0.2_wall_albedo_0.2_pav_albedo_0.15_traffic_heat_4_' is generated at data.

Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0_tree_cov_0.15_glz_ratio_0.4_roof_albedo_0.2_wall_albedo_0.2_pav_albedo_0.15_traffic_heat_2_' is generated at data.

Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0_tree_cov_0.15_glz_ratio_0.4_roof_albedo_0.2_wall_albedo_0.2_pav_albedo_0.2_traffic_heat_4_' is generated at data.

Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0_tree_cov_0.15_glz_ratio_0.4_roof_albedo_0.2_wall_albedo_0.2_pav_albedo_0.2_traffic_heat_2_' is generated at data.

Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0_tree_cov_0.15_glz_ratio_0.4_roof

...


Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0.1_tree_cov_0.15_glz_ratio_0.2_roof_albedo_0.2_wall_albedo_0.25_pav_albedo_0.2_traffic_heat_4_' is generated at data.

Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0.1_tree_cov_0.15_glz_ratio_0.2_roof_albedo_0.2_wall_albedo_0.25_pav_albedo_0.2_traffic_heat_2_' is generated at data.

Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0.1_tree_cov_0.15_glz_ratio_0.4_roof_albedo_0.2_wall_albedo_0.15_pav_albedo_0.1_traffic_heat_4_' is generated at data.

Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0.1_tree_cov_0.15_glz_ratio_0.4_roof_albedo_0.2_wall_albedo_0.15_pav_albedo_0.1_traffic_heat_2_' is generated at data.

Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0.1_tree_cov_0.15_glz_ra


Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0.3_tree_cov_0.05_glz_ratio_0.2_roof_albedo_0.2_wall_albedo_0.15_pav_albedo_0.1_traffic_heat_4_' is generated at data.

Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0.3_tree_cov_0.05_glz_ratio_0.2_roof_albedo_0.2_wall_albedo_0.15_pav_albedo_0.1_traffic_heat_2_' is generated at data.

Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0.3_tree_cov_0.05_glz_ratio_0.2_roof_albedo_0.2_wall_albedo_0.15_pav_albedo_0.15_traffic_heat_4_' is generated at data.

Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0.3_tree_cov_0.05_glz_ratio_0.2_roof_albedo_0.2_wall_albedo_0.15_pav_albedo_0.15_traffic_heat_2_' is generated at data.

Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0.3_tree_cov_0.05_glz_


Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0.3_tree_cov_0.1_glz_ratio_0.3_roof_albedo_0.2_wall_albedo_0.15_pav_albedo_0.15_traffic_heat_4_' is generated at data.

Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0.3_tree_cov_0.1_glz_ratio_0.3_roof_albedo_0.2_wall_albedo_0.15_pav_albedo_0.15_traffic_heat_2_' is generated at data.

Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0.3_tree_cov_0.1_glz_ratio_0.3_roof_albedo_0.2_wall_albedo_0.15_pav_albedo_0.2_traffic_heat_4_' is generated at data.

Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0.3_tree_cov_0.1_glz_ratio_0.3_roof_albedo_0.2_wall_albedo_0.15_pav_albedo_0.2_traffic_heat_2_' is generated at data.

Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0.3_tree_cov_0.1_glz_ratio


Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0.3_tree_cov_0.1_glz_ratio_0.4_roof_albedo_0.2_wall_albedo_0.15_pav_albedo_0.2_traffic_heat_4_' is generated at data.

Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0.3_tree_cov_0.1_glz_ratio_0.4_roof_albedo_0.2_wall_albedo_0.15_pav_albedo_0.2_traffic_heat_2_' is generated at data.

Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0.3_tree_cov_0.1_glz_ratio_0.4_roof_albedo_0.2_wall_albedo_0.2_pav_albedo_0.1_traffic_heat_4_' is generated at data.

Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0.3_tree_cov_0.1_glz_ratio_0.4_roof_albedo_0.2_wall_albedo_0.2_pav_albedo_0.1_traffic_heat_2_' is generated at data.

Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0.3_tree_cov_0.1_glz_ratio_0.4


Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0.3_tree_cov_0.15_glz_ratio_0.2_roof_albedo_0.2_wall_albedo_0.2_pav_albedo_0.1_traffic_heat_4_' is generated at data.

Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0.3_tree_cov_0.15_glz_ratio_0.2_roof_albedo_0.2_wall_albedo_0.2_pav_albedo_0.1_traffic_heat_2_' is generated at data.

Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0.3_tree_cov_0.15_glz_ratio_0.2_roof_albedo_0.2_wall_albedo_0.2_pav_albedo_0.15_traffic_heat_4_' is generated at data.

Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0.3_tree_cov_0.15_glz_ratio_0.2_roof_albedo_0.2_wall_albedo_0.2_pav_albedo_0.15_traffic_heat_2_' is generated at data.

Simulating new temperature and humidity values for 365 days from 1/1.

New climate file 'grass_cov_0.3_tree_cov_0.15_glz_rati